In [2]:
import numpy as np
import scipy.io as sio
import glob
from sklearn import linear_model
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import math
from itertools import cycle
from scipy.optimize import curve_fit
from sklearn.decomposition import PCA
from scipy.stats import sem
from scipy import stats
from scipy import ndimage
import pystan
from matplotlib import cm

%matplotlib inline

color_cycle = cycle(['g', 'b', 'c', 'm', 'y', 'k'])


import warnings
warnings.simplefilter("ignore")


import matplotlib as mpl
mpl.rcParams['font.size'] = 16
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'large'

%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

color_cycle = cycle(['g', 'b', 'c', 'm', 'y', 'k'])


import sys
sys.path.append('C:\\Users\\adamimos\\Documents\\GitHub\\ASAC\\python')
from import_rat_data import *
from behavior_analysis import *
#from decoder_functions import *

def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import cv2

In [1]:
# LOAD ALL DATA

directory = 'H:\\adam0_20190731\\'
#directory = 'G:\\centerhold_20190712\\'
#directory = 'G:\\adam0_20190705\\'
behavior_files = glob.glob(directory+'*.pickle')
frame_files = glob.glob(directory+'*.csv')

behavior_data = []
frame_data = []

for b in behavior_files:
    file = open(b,'rb')
    behavior_data.append(pickle.load(file))
    file.close()   

for f in frame_files:
    frame_data.append(get_frame_data(f))

try:
    traces = sio.loadmat(glob.glob(directory+'*traces.mat')[0])['traces']
except:
    traces = sio.loadmat(glob.glob(directory+'*traces.mat')[0])['traces_']

NameError: name 'glob' is not defined

In [ ]:
behavior_data[0]['correct_side']

In [8]:
was_completed = []
was_correct = []
stim_noise = []
correct_side = []
stim_dir = []
response_side = []
prior = []
spf = []
ssf = []
sef = []
rf = []
stim_type = []
pattern_inds = []
patterns = []

frame_offset = np.cumsum(0)
frame_offset = np.roll(frame_offset,1)
frame_offset[0]=0
for i in range(len(behavior_data)):
    bd = behavior_data[i]
    fd = frame_data[i]
    ft = bd['final_trial']+1
    
    #ft = np.where([x==0 for x in bd['stim_dir']])[0][0] # FIX FOR CENTER HOLD
    
    was_completed.extend(bd['was_completed'][0:ft])
    was_correct.extend(bd['was_correct'][0:ft])
    stim_noise.extend(bd['coherence'][0:ft])
    #stim_type.extend(bd['stim_type'][0:ft])
    stim_dir.extend(bd['stim_dir'][0:ft])
    correct_side.extend(bd['correct_side'][0:ft])
    response_side.extend(bd['response_side'][0:ft])
    prior.extend(bd['rightward_prior'][0:ft])
    ssf.extend(fd['stim_start_frames']+frame_offset[i])
    spf.extend(fd['start_poke_frames']+frame_offset[i])
    sef.extend(fd['stim_end_frames']+frame_offset[i])
    rf.extend(fd['response_frames']+frame_offset[i])
    #pattern_inds.extend(bd['pattern_inds_played'][0:ft])
    #patterns.extend(bd['patterns'])
    

In [9]:
traces_stim_aligned = []
traces_resp_aligned = []
completed_inds = []


k = 0
for ind in range(len(was_completed)):
    if was_completed[ind]==1:
    #if not np.isnan(rf[ind]) and not np.isnan(ssf[ind]): # CAHNGED FOR CENTER HOLD
        k=k+1
        traces_stim_aligned.append(traces[:,np.arange(ssf[ind]-40,ssf[ind]+80,1).astype(int)])
        traces_resp_aligned.append(traces[:,np.arange(rf[ind]-60,rf[ind]+60,1).astype(int)])
        completed_inds.append(ind)
        
traces_stim_aligned = np.array(traces_stim_aligned)
traces_resp_aligned = np.array(traces_resp_aligned)

completed_inds = np.array(completed_inds)

In [10]:
all_data = {'was_completed':np.array(was_completed),
           'was_correct':np.array(was_correct),
           'stim_dir':np.array(stim_dir),
           'correct_side':np.array(correct_side),
           'response_side':np.array(response_side),
            'prior':np.array(prior),
            'noise':np.array(stim_noise),
            #'stim_type':np.array(stim_type),
           'traces':traces,
           'task_info':behavior_data[0]['info'],
           'frame_info':{'stim_start_frame':np.array(ssf),
                   'start_poke_frame':np.array(spf),
                   'stim_end_frame':np.array(sef),
                   'response_frame':np.array(rf)},
            #'patterns':np.array(patterns),
            #'pattern_inds':np.array(pattern_inds)
           }

ci = np.where(np.array(was_completed))
ci = completed_inds # CHANGED FOR CENTER HOLD
completed_trials_data = {'was_completed':np.array(was_completed)[ci],
           'was_correct':np.array(was_correct)[ci],
           'stim_dir':np.array(stim_dir)[ci],
           'correct_side':np.array(correct_side)[ci],
            'prior':np.array(prior)[ci],
            'noise':np.array(stim_noise)[ci],
            #'stim_type':np.array(stim_type)[ci],
           'response_side':np.array(response_side)[ci],
            'traces_stim_aligned':traces_stim_aligned,
            'traces_resp_aligned':traces_resp_aligned,
            'completed_inds':ci,
            'frame_info':{'stim_start_frame':np.array(ssf)[ci],
                   'start_poke_frame':np.array(spf)[ci],
                   'stim_end_frame':np.array(sef)[ci],
                   'response_frame':np.array(rf)[ci]},
            #        'patterns':np.array(patterns),
            #'pattern_inds':np.array(pattern_inds)[ci]
            }

data = {'all_data':all_data,'completed_trials_data':completed_trials_data}

pickle.dump( data, open( "adam0_20190731.pickle", "wb" ) )